각 test_ID에 교통사고 값(ECLO)을 예측하기 

ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1

- 출퇴근 시간, 새벽 시간(음주, 졸음)에 많이 발생?

- 요일에 따라 달라진다? -> 인구 밀집

- 날씨 영향?

- 사고가 빈번한 지역?

- cctv가 없거나 신호 시간이 짧거나

- 도로 상태, 형태

- 노인이 많으면 사고가 자주 발생

# 카테코리로 묶어 xgb, lgbm 실행하기

# 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# 데이터 불러오기

In [3]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
accident_df = pd.read_csv('./data/external_open/countrywide_accident.csv')

train_df.head(3)
train_df.shape

test_df.head(3)
test_df.shape

accident_df.head(3)
accident_df.shape

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3


(39609, 23)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차


(10963, 8)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01 00,화요일,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,상해없음,보행자,남,40세,경상,0,0,1,0,3
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01 00,화요일,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,상해없음,이륜,남,20세,경상,0,0,1,0,3
2,COUNTRYWIDE_ACCIDENT_000002,2019-01-01 00,화요일,맑음,경기도 양주시 고암동,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,상해없음,이륜,남,17세,중상,0,1,0,0,5


(602775, 23)

In [4]:
# 추가할 수 있는 data

cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv', encoding = 'cp949')
security_light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv', encoding = 'cp949', low_memory = False)
child_protection_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv', encoding = 'cp949')
parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv', encoding = 'cp949')

# cctv_df.head(2)
# security_light_df.head(2)
# child_protection_df.head(2)
# parking_df.head(2)

# EDA

## 결측치 확인

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

=> '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'에 약 1000개의 결측치 존재

In [6]:
test_df.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    10963 non-null  object
 4   시군구     10963 non-null  object
 5   도로형태    10963 non-null  object
 6   노면상태    10963 non-null  object
 7   사고유형    10963 non-null  object
dtypes: object(8)
memory usage: 685.3+ KB


In [7]:
accident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602775 entries, 0 to 602774
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           602775 non-null  object
 1   사고일시         602775 non-null  object
 2   요일           602775 non-null  object
 3   기상상태         602775 non-null  object
 4   시군구          602775 non-null  object
 5   도로형태         602775 non-null  object
 6   노면상태         602774 non-null  object
 7   사고유형         602775 non-null  object
 8   사고유형 - 세부분류  602775 non-null  object
 9   법규위반         602775 non-null  object
 10  가해운전자 차종     602775 non-null  object
 11  가해운전자 성별     602775 non-null  object
 12  가해운전자 연령     602775 non-null  object
 13  가해운전자 상해정도   602775 non-null  object
 14  피해운전자 차종     576948 non-null  object
 15  피해운전자 성별     576946 non-null  object
 16  피해운전자 연령     576949 non-null  object
 17  피해운전자 상해정도   576946 non-null  object
 18  사망자수         602775 non-null  int64 
 19  중상

=> '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'에 결측치가 존재하는 이유 -> '사고 유형'이 '챠랑단독'이기 때문이다

=> 이런 결측치는 어떻게 처리를 해야하나

# 데이터 전처리

## train에 있고 test에 없는 값 삭제

In [8]:
train_df['기상상태'].unique()
accident_df['기상상태'].unique()
test_df['기상상태'].unique()

array(['맑음', '흐림', '기타', '비', '안개', '눈'], dtype=object)

array(['맑음', '흐림', '눈', '기타', '비', '안개'], dtype=object)

array(['맑음', '흐림', '기타', '비', '눈'], dtype=object)

In [9]:
train_df['노면상태'].unique()
accident_df['노면상태'].unique()
test_df['노면상태'].unique()

array(['건조', '젖음/습기', '서리/결빙', '기타', '침수', '적설'], dtype=object)

array(['건조', '젖음/습기', '기타', '서리/결빙', '적설', '해빙', '침수', nan], dtype=object)

array(['건조', '젖음/습기', '서리/결빙', '기타', '침수', '적설'], dtype=object)

In [10]:
train_df['사고유형'].unique()
accident_df['사고유형'].unique()
test_df['사고유형'].unique()

array(['차대사람', '차대차', '차량단독'], dtype=object)

array(['차대사람', '차대차', '차량단독', '철길건널목'], dtype=object)

array(['차대사람', '차대차', '차량단독'], dtype=object)

=> train, accident '기상상태'에는 안개가 있고 test '기상상태'에는 안개가 없어 삭제

=> accident '노면상태'에는 해빙, nan가 있고 train test에는 없어 삭제

=> accident '사고유형'에는 철길건널목이 있고 train, test에는 없어 삭제

In [11]:
# '기상상태'에서 '안개' 삭제

train_df = train_df[train_df['기상상태'] != '안개']
accident_df = accident_df[accident_df['기상상태'] != '안개']

train_df['기상상태'].unique()
accident_df['기상상태'].unique()

array(['맑음', '흐림', '기타', '비', '눈'], dtype=object)

array(['맑음', '흐림', '눈', '기타', '비'], dtype=object)

In [12]:
# '노면상태'에서 '해빙', nan 삭제

accident_df = accident_df[accident_df['노면상태'] != '해빙']
accident_df = accident_df[~accident_df['노면상태'].isnull()]

accident_df['노면상태'].unique()

array(['건조', '젖음/습기', '기타', '서리/결빙', '적설', '침수'], dtype=object)

In [13]:
# '사고유형'에서 '철길건널목' 삭제

accident_df = accident_df[accident_df['사고유형'] != '철길건널목']

accident_df['사고유형'].unique()

array(['차대사람', '차대차', '차량단독'], dtype=object)

## 시군구, 날짜, 도로형태, 사고 유형 분리 분리하기

In [14]:
# accident_df에 '시군구'를 나눌 때 결측치가 발생하므로 따로 처리
for i in range(len(accident_df)):
    name = accident_df.iloc[i]['시군구']
    name_lst = name.split()

    if len(name_lst) == 2:
        re_name = f'{name_lst[0]}{name_lst[0]}{name_lst[0]}'
        accident_df.loc[i, '시군구'] = re_name

In [15]:
# 시군구 나누는 함수 구현

def si_gun_gu(df):
    lst = df['시군구'].str.split(' ')
    df['시'] = lst.str[0]
    df['군'] = lst.str[1]
    df['구'] = lst.str[2]
    return df

In [16]:
train_df = si_gun_gu(train_df)
test_df = si_gun_gu(test_df)
accident_df = si_gun_gu(accident_df)

train_df.head(3)
test_df.head(3)
accident_df.head(3)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,시,군,구
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,70세,중상,0,1,0,0,5,대구광역시,중구,대신동
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,61세,경상,0,0,1,0,3,대구광역시,달서구,감삼동
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,38세,경상,0,0,1,0,3,대구광역시,수성구,두산동


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,시,군,구
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,대구광역시,수성구,상동
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,대구광역시,수성구,지산동
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,대구광역시,수성구,수성동2가


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,시,군,구
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01 00,화요일,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,40세,경상,0.0,0.0,1.0,0.0,3.0,서울특별시,강서구,방화동
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01 00,화요일,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,20세,경상,0.0,0.0,1.0,0.0,3.0,경기도,포천시,소흘읍
2,COUNTRYWIDE_ACCIDENT_000002,2019-01-01 00,화요일,맑음,경기도 양주시 고암동,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,17세,중상,0.0,1.0,0.0,0.0,5.0,경기도,양주시,고암동


In [17]:
# 사고일시를 나누는 함수 

def year_mon_day(df):
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    df['연'] = df['사고일시'].dt.year
    df['월'] = df['사고일시'].dt.month
    df['일'] = df['사고일시'].dt.day
    df['요일'] = df['사고일시'].dt.day_of_week # 0:월 ~ 6일요일
    df['시간'] = df['사고일시'].dt.hour
    return df

In [18]:
train_df = year_mon_day(train_df)
test_df = year_mon_day(test_df)
accident_df = year_mon_day(accident_df)

train_df.head(3)
test_df.head(3)
accident_df.head(3)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,경상자수,부상자수,ECLO,시,군,구,연,월,일,시간
0,ACCIDENT_00000,2019-01-01 00:00:00,1,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,0,0,5,대구광역시,중구,대신동,2019,1,1,0
1,ACCIDENT_00001,2019-01-01 00:00:00,1,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,1,0,3,대구광역시,달서구,감삼동,2019,1,1,0
2,ACCIDENT_00002,2019-01-01 01:00:00,1,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,1,0,3,대구광역시,수성구,두산동,2019,1,1,1


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,시,군,구,연,월,일,시간
0,ACCIDENT_39609,2022-01-01 01:00:00,5,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,대구광역시,수성구,상동,2022,1,1,1
1,ACCIDENT_39610,2022-01-01 01:00:00,5,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,대구광역시,수성구,지산동,2022,1,1,1
2,ACCIDENT_39611,2022-01-01 04:00:00,5,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,대구광역시,수성구,수성동2가,2022,1,1,4


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,경상자수,부상자수,ECLO,시,군,구,연,월,일,시간
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01,1.0,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,1.0,0.0,3.0,서울특별시,강서구,방화동,2019.0,1.0,1.0,0.0
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01,1.0,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,1.0,0.0,3.0,경기도,포천시,소흘읍,2019.0,1.0,1.0,0.0
2,COUNTRYWIDE_ACCIDENT_000002,2019-01-01,1.0,맑음,경기도 양주시 고암동,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,0.0,0.0,5.0,경기도,양주시,고암동,2019.0,1.0,1.0,0.0


In [19]:
# 도로형태를 '-'를 기준으로 분리하는 함수 구현
def road(df):
    lst = df['도로형태'].str.split(' - ')
    df['도로형태 - 대분류'] = lst.str[0]
    df['도로형태 - 소분류'] = lst.str[1]
    return df

In [20]:
train_df = road(train_df)
test_df = road(test_df)
accident_df = road(accident_df)

train_df.head(2)
test_df.head(2)
accident_df.head(2)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,ECLO,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류
0,ACCIDENT_00000,2019-01-01,1,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,5,대구광역시,중구,대신동,2019,1,1,0,단일로,기타
1,ACCIDENT_00001,2019-01-01,1,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,3,대구광역시,달서구,감삼동,2019,1,1,0,단일로,기타


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류
0,ACCIDENT_39609,2022-01-01 01:00:00,5,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,대구광역시,수성구,상동,2022,1,1,1,교차로,교차로안
1,ACCIDENT_39610,2022-01-01 01:00:00,5,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,대구광역시,수성구,지산동,2022,1,1,1,단일로,기타


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,ECLO,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01,1.0,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,3.0,서울특별시,강서구,방화동,2019.0,1.0,1.0,0.0,교차로,교차로횡단보도내
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01,1.0,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,3.0,경기도,포천시,소흘읍,2019.0,1.0,1.0,0.0,교차로,교차로안


In [22]:
# 사고 유형 분리하기
# 사고 유형은 차대차, 차대 사람, 차량단독
def car_people(df):
    pattern = r'(\S)(\S)(\S+)'
    df[['사고유형1', '대', '사고유형2']] = df['사고유형'].str.extract(pattern)

    return df

In [23]:
train_df = car_people(train_df)
test_df = car_people(test_df)
accident_df = car_people(accident_df)

In [24]:
# 필요없는 col 제거
train_df = train_df.drop(['사고일시', '시군구', '도로형태', '사고유형', '대'], axis = 1)
test_df = test_df.drop(['사고일시', '시군구', '도로형태', '사고유형', '대'], axis = 1)
accident_df = accident_df.drop(['사고일시', '시군구', '도로형태', '사고유형', '대'], axis = 1)

In [25]:
train_df.head(1)
test_df.head(1)

,ID,요일,기상상태,노면상태,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류,사고유형1,사고유형2
0,ACCIDENT_00000,1,맑음,건조,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,...,중구,대신동,2019,1,1,0,단일로,기타,차,사람


,ID,요일,기상상태,노면상태,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류,사고유형1,사고유형2
0,ACCIDENT_39609,5,맑음,건조,대구광역시,수성구,상동,2022,1,1,1,교차로,교차로안,차,사람


In [26]:
train_df['사고유형1'].unique()
train_df['사고유형2'].unique()

array(['차'], dtype=object)

array(['사람', '차', '단독'], dtype=object)

## 연령을 수치로

In [ ]:
# train_df['가해운전자 연령'] = train_df['가해운전자 연령'].str.replace('세', '')
# train_df['가해운전자 연령'] = train_df['가해운전자 연령'].astype('int')
# train_df['가해운전자 연령']

In [ ]:
# train_df['가해운전자 연령'].unique()

=> 가해운전자 연량에 미분류(900개), 18, 17, 16, 10, 13, 14, 15, 11, 9, 4, 8, 12, ,7, 5는 흠,,

=> 90 이상, 98 이상, 90은 그냥 90이상으로 변경

In [ ]:
# train_df[train_df['가해운전자 연령'] == '5'].T
# train_df[train_df['가해운전자 연령'] == '미분류'].iloc[1].T

In [ ]:
# train_df[train_df['가해운전자 연령'] == '미분류'].loc[:, '사고일시' : '가해운전자 상해정도']

# 모델링

In [28]:
X_test = test_df.drop(['ID'], axis = 1).copy()
X_train = train_df[X_test.columns].copy()
y_train = train_df['ECLO'].copy()

In [29]:
X_test.head(1)
X_train.head(1)
y_train.head(1)

,요일,기상상태,노면상태,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류,사고유형1,사고유형2
0,5,맑음,건조,대구광역시,수성구,상동,2022,1,1,1,교차로,교차로안,차,사람


,요일,기상상태,노면상태,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류,사고유형1,사고유형2
0,1,맑음,건조,대구광역시,중구,대신동,2019,1,1,0,단일로,기타,차,사람


0    5
Name: ECLO, dtype: int64

In [30]:
# LGBMRegressor 파라미터

cat_features = list(X_train.dtypes[X_train.dtypes == 'object'].index)
cat = CatBoostRegressor(
    learning_rate=0.1,          # 학습률
    cat_features = cat_features,
    random_state = 42, 
    verbose = False,
    iterations=100,             # 반복 횟수 (트리 개수)
    depth=6,                    # 트리의 최대 깊이
    l2_leaf_reg=3,              # L2 규제 가중치
    loss_function='RMSE'        # 손실 함수 (회귀 문제의 경우 RMSE)
)

In [31]:
# KFold이용

cv = KFold(n_splits = 10, shuffle = True, random_state = 42)
score_lst = []
cat_lst = []

for train_idx, test_idx in tqdm(cv.split(X_train), total = 10):
    x_tr = X_train.iloc[train_idx]
    y_tr = y_train.iloc[train_idx]
    
    x_val = X_train.iloc[test_idx]
    y_val = y_train.iloc[test_idx]
    
    model = cat
    model.fit(x_tr, y_tr)

    pred = model.predict(x_val)
    score = mean_squared_error(y_val, pred, squared = False)
    score_lst.append(score)
    
#     cat_pred = np.expm1(model.fit(x_val))
#     cat_pred = np.where(cat_pred < 1, 1, cat_pred)
#     cat_lst.append(cat_pred)

print('each fold rmse : ', score_lst)
print('rmse : ', np.mean(score_lst))

  0%|                                                    | 0/10 [00:00<?, ?it/s]

 10%|████▍                                       | 1/10 [00:00<00:04,  1.98it/s]

 20%|████████▊                                   | 2/10 [00:00<00:03,  2.17it/s]

 30%|█████████████▏                              | 3/10 [00:01<00:03,  2.25it/s]

 40%|█████████████████▌                          | 4/10 [00:01<00:02,  2.28it/s]

...

 70%|██████████████████████████████▊             | 7/10 [00:03<00:01,  2.30it/s]

 80%|███████████████████████████████████▏        | 8/10 [00:03<00:00,  2.32it/s]

 90%|███████████████████████████████████████▌    | 9/10 [00:03<00:00,  2.33it/s]

100%|███████████████████████████████████████████| 10/10 [00:04<00:00,  2.28it/s]

each fold rmse :  [3.137898279927288, 3.284512436396484, 3.2307242879876372, 2.96605919440436, 3.2755339961672303, 3.187659707485155, 3.2739613136431314, 3.0286707171871856, 2.9553126977528508, 3.155943710281724]
rmse :  3.1496276341233047
